In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import seaborn as sns
import math
from datetime import date, timedelta

%matplotlib inline
from matplotlib import font_manager, rc

In [ ]:
card = pd.read_csv('KT_data_20200703/card.csv')
card = card[['receipt_dttm', 'adstrd_nm', 'mrhst_induty_cl_nm', 'selng_cascnt', 'salamt']]
card = card[card['selng_cascnt'].apply(lambda x: x.isnumeric())]
card = card[card['salamt'].apply(lambda x: x.isnumeric())]
card['selng_cascnt'] = card['selng_cascnt'].astype(int)
card['salamt'] = card['salamt'].astype(int)
card['receipt_dttm'] = card['receipt_dttm'].astype(str)

# 일별, 업종별 카드사용금액
day = card.groupby(['receipt_dttm','mrhst_induty_cl_nm'])['salamt'].sum().reset_index()
daily_sale_amount = day.pivot(index='mrhst_induty_cl_nm', columns='receipt_dttm', values='salamt')

for index, row in daily_sale_amount.iterrows():
    max_salamt = 0
    for col in daily_sale_amount.columns:
        if daily_sale_amount.loc[index,col] < 0 or math.isnan(daily_sale_amount.loc[index, col]):
            daily_sale_amount.loc[index, col] = 0
        if row.loc[col] > max_salamt:
            max_salamt = float(daily_sale_amount.loc[index, col])

    # Normalized by the max sale amount
    for col in daily_sale_amount.columns:
        if max_salamt != 0:
            daily_sale_amount.loc[index, col] = float(daily_sale_amount.loc[index, col]) / float(max_salamt)
        if max_salamt == 0:  # All salamts are negative or nan values
            daily_sale_amount.loc[index, col] = 0

In [ ]:
clustergrid = sns.clustermap(daily_sale_amount, col_cluster=False, cmap='Blues', figsize=(12, 14));

# Clustering 결과 확인 후 Cluster를 네모 박스로 표시
sub_clusters = []
sub_clusters.append(('국산 신차', '홍삼 제품', 'r'))
sub_clusters.append(('초중고교육기관', '카지노', 'r'))
sub_clusters.append(('유흥주점', '사진관', 'r'))
sub_clusters.append(('한의원', '화물 운송', 'r'))
sub_clusters.append(('대형할인점', '한정식', 'r'))
sub_clusters.append(('중고자동차', '화방표구점', 'r'))

ax = clustergrid.ax_heatmap
xmax = len(daily_sale_amount.axes[1])
rows = list(daily_sale_amount.axes[0])
reordered_row = [rows[x] for x in clustergrid.dendrogram_row.reordered_ind]

for start, end, color in sub_clusters:
    # vertical lines
    ax.plot([0, 0], [reordered_row.index(start), reordered_row.index(end)+1], color, lw = 2.5)
    ax.plot([xmax-.1, xmax-.1], [reordered_row.index(start), reordered_row.index(end)+1], color, lw = 2.5)
    # horizontal lines
    ax.plot([0, xmax-.1], [reordered_row.index(start), reordered_row.index(start)], color, lw = 2.5)
    ax.plot([0, xmax-.1], [reordered_row.index(end)+1, reordered_row.index(end)+1], color, lw = 2.5)
ax.set_xlabel("날짜", fontsize=13);
ax.set_ylabel("업종", fontsize=13);
plt.ylabel('최고 소비금액 대비 비율');